In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D

# ==========================================
# 1. PREPROCESSING & DATASET
# ==========================================
# Load data
df = pd.read_csv('accelerometer.csv')

# Labeling: 1 -> Normal (0), Lainnya -> Anomaly (1)
df['label'] = df['wconfid'].apply(lambda x: 0 if x == 1 else 1)

# Normalisasi Fitur (x, y, z)
scaler = StandardScaler()
df[['x', 'y', 'z']] = scaler.fit_transform(df[['x', 'y', 'z']])

# Fungsi Windowing
def create_sequences(X, y, time_steps=60, step=30):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        # Label diambil dari mode (mayoritas) di window tersebut
        ys.append(y.iloc[i:(i + time_steps)].mode()[0])
    return np.array(Xs), np.array(ys)

X_data = df[['x', 'y', 'z']]
y_data = df['label']

X_windows, y_windows = create_sequences(X_data, y_data, time_steps=60, step=30)
print(f"Shape Data Input: {X_windows.shape}") # (Jumlah_Sampel, 60, 3)

# Split Train (70%), Val (15%), Test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_windows, y_windows, test_size=0.3, random_state=42, stratify=y_windows
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

Shape Data Input: (5098, 60, 3)


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D

# ==========================================
# 2. ARSITEKTUR HYBRID CNN-LSTM
# ==========================================
model = Sequential([
    # CNN Layer: Ekstraksi Fitur
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(60, 3)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2), # Regularisasi

    # LSTM Layer: Belajar pola waktu
    LSTM(50),
    Dropout(0.2),

    # Output Layer
    Dense(1, activation='sigmoid') # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 58, 64)         │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,691 (92.54 KB)

 Trainable params: 23,691 (92.54 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D

# ==========================================
# 3. PELATIHAN (TRAINING)
# ==========================================
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6751 - loss: 0.6278 - val_accuracy: 0.8078 - val_loss: 0.4003
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8254 - loss: 0.4095 - val_accuracy: 0.9098 - val_loss: 0.3189
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.8802 - loss: 0.3006 - val_accuracy: 0.9150 - val_loss: 0.2360
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8895 - loss: 0.2562 - val_accuracy: 0.9124 - val_loss: 0.1920
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9159 - loss: 0.2056 - val_accuracy: 0.9359 - val_loss: 0.1751
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9260 - loss: 0.1778 - val_accuracy: 0.9438 - val_loss: 0.1407
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9184 - loss: 0.2054 - val_accuracy: 0.9242 - val_loss: 0.1855
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9202 - loss: 0.1838 - val_accuracy: 0.9660 - v

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D

# ==========================================
# 4. EVALUASI
# ==========================================
# Prediksi CNN-LSTM
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\n--- Evaluasi CNN-LSTM ---")
print(classification_report(y_test, y_pred))
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_prob):.4f}")

# Baseline: Random Forest (sebagai pembanding)
# Data harus di-flatten (didatarkan) untuk masuk ke algoritma ML klasik
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train)
y_pred_rf = rf.predict(X_test_flat)

print("\n--- Evaluasi Baseline (Random Forest) ---")
print(classification_report(y_test, y_pred_rf))

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step

--- Evaluasi CNN-LSTM ---
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       255
           1       0.99      0.97      0.98       510

    accuracy                           0.97       765
   macro avg       0.97      0.97      0.97       765
weighted avg       0.97      0.97      0.97       765

ROC-AUC Score: 0.9946

--- Evaluasi Baseline (Random Forest) ---
              precision    recall  f1-score   support

           0       0.84      0.76      0.79       255
           1       0.88      0.93      0.90       510

    accuracy                           0.87       765
   macro avg       0.86      0.84      0.85       765
weighted avg       0.87      0.87      0.87       765



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D

# ==========================================
# 5. DEPLOYMENT (Export ke TFLite)
# ==========================================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Opsional: Optimasi untuk ukuran (Quantization)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Mengatasi error pada LSTM saat konversi:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # Ops standar TFLite
  tf.lite.OpsSet.SELECT_TF_OPS    # Tambahkan TF Ops jika LSTM kompleks
]
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

with open('model_system.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model berhasil diekspor ke format TFLite.")

Saved artifact at '/tmp/tmp22qedd54'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 3), dtype=tf.float32, name='keras_tensor_7')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134857545644560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545642064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545641872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545645328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545643216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545643024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134857545642448: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model berhasil diekspor ke format TFLite.
